In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import plotly_express as px

# 1. Objectif

Le jeu de données sur lequel nous sommes amenés à travailler est assez conséquent (5.9 GB) et donc pour **faciliter l'analyse exploratoire des données** notre premier objectif dés de retirer toutes les colonnes qui ne sont d'aucune utilité car:
- vides
- disposant de trop peu de données
- redondantes avec d'autres colonnes
- ne présentant pas d'intérêt

# 2. Présentation générale du jeu de données <a class="anchor" id="P02"></a>

Le [jeu de données](https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv) utilisé dans le cadre de l'appel à projets de l'agence *Santé publique France* qui consiste à rendre les données de santé plus accessibles, est une liste de 2.251.894 produits alimentaires répertoriés par les volontaires de l'association [Open Food Facts](https://world.openfoodfacts.org).

Chacun des produits référencé est décrit par un certain nombre de caractèristiques nutritionels *(taux de graisse, de sucre, de sel, de fibres, de protéines, de vitamines, etc.)* et par des méta-données *(code-barre, nom du produit, catégorie, lieu de production, data d'ajout dans la DB, auteur de l'ajout, etc.)*.

---
#### Commençons par importer et explorer simplement ce jeu de données pour nous faire une idée de ce qu'il contient.

In [2]:
filename = 'data/en.openfoodfacts.org.products.csv'

#### Affichons uniquement la première ligne

In [3]:
prob = pd.read_csv(filename, sep='\t', low_memory=True, nrows=1)
prob

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,jeunes pousses,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Regardons l'ensemble des colonnes présentes *(et leur index pour pouvoir ajuster leur type en cas de mixed-type)*

In [4]:
size = max([len(x) for x in prob.columns])-2

for i, col in enumerate(prob.columns):
    end_sep="\t" if i%2 == 0 else "\n"
    print(f"{i} {col[:size].ljust(size)}", end=end_sep)

0 code                                               	1 url                                                
2 creator                                            	3 created_t                                          
4 created_datetime                                   	5 last_modified_t                                    
6 last_modified_datetime                             	7 product_name                                       
8 abbreviated_product_name                           	9 generic_name                                       
10 quantity                                           	11 packaging                                          
12 packaging_tags                                     	13 packaging_en                                       
14 packaging_text                                     	15 brands                                             
16 brands_tags                                        	17 categories                                         
18 categories_tags  

# 3. Identification des colonnes que l'on ne souhaite pas garder

#### Chargeons le CSV par petit bouts pour en calculer le `taux de remplissage` puis identifier les colonnes vides ou presque vides.

Nous allons également fournir les types pour les colonnes dont pandas n'est pas certain.

In [5]:
dtypes = {}
for col_name in prob.columns:
    if col_name.endswith(('_100g', '_quantity')):
        dtype = 'float'
    elif col_name.endswith(('_t')): # , '_n' <- probleme avec les NA
        dtype = 'int' 
    else:
        dtype = 'str'
        
    dtypes[col_name] = dtype

In [11]:
chunksize = 10000
nan_rates = []
for chunk in pd.read_csv(filename, chunksize=chunksize, sep='\t', low_memory=True, dtype=dtypes):
    nan_rates.append(chunk.notnull().mean(axis=0))

On fait la moyenne des taux calculés sur tous les morceaux

In [12]:
fill_ratio = pd.concat(nan_rates, axis=1).mean(axis=1)

In [40]:
size = max([len(x) for x in prob.columns])-13

for i, ratio in enumerate(fill_ratio.iteritems()):
    end_sep="\t" if i%2 == 0 else "\n"
    print(f"{i:3} {ratio[0][:size].ljust(size)} {ratio[1]*100:7.2f}%", end=end_sep)

  0 code                                      100.00%	  1 url                                       100.00%
  2 creator                                   100.00%	  3 created_t                                 100.00%
  4 created_datetime                          100.00%	  5 last_modified_t                           100.00%
  6 last_modified_datetime                    100.00%	  7 product_name                               96.21%
  8 abbreviated_product_name                    0.35%	  9 generic_name                                5.62%
 10 quantity                                   25.84%	 11 packaging                                  14.77%
 12 packaging_tags                             14.77%	 13 packaging_en                               14.77%
 14 packaging_text                              0.49%	 15 brands                                     48.94%
 16 brands_tags                                48.94%	 17 categories                                 44.74%
 18 categories_tags         

Et on l'affiche sous forme de graphique

In [41]:
fill_ratio_df = pd.DataFrame(fill_ratio*100, columns=['fill_rate'])
#fill_ratio_df = fill_ratio_df.sort_values(by='fill_rate', ascending=False)

fig = px.bar(fill_ratio_df, y="fill_rate", 
             #width=900,
             height=600,
             color='fill_rate', 
             title="Taux moyen de remplissage des colonnes",
             labels={
                "fill_rate": "Remplissage (en %)",
                "index": "",
                },
            )
fig.update_coloraxes(showscale=False)
fig.update_xaxes(tickangle = -45)
fig.show()

> On constate qu'il y a de nombreuses colonnes totalement vides, et d'autres qui n'ont que très peu de données.

#### Supprimons les colonnes vides

In [97]:
cols_fillrate_00 = fill_ratio[fill_ratio == 0.00].index

for col_name in sorted(cols_fillrate_00):
    print(col_name, end=" | ")
    
len(cols_fillrate_00)

-alpha-linolenic-acid_100g | -arachidic-acid_100g | -arachidonic-acid_100g | -behenic-acid_100g | -butyric-acid_100g | -capric-acid_100g | -caproic-acid_100g | -caprylic-acid_100g | -cerotic-acid_100g | -dihomo-gamma-linolenic-acid_100g | -docosahexaenoic-acid_100g | -eicosapentaenoic-acid_100g | -elaidic-acid_100g | -erucic-acid_100g | -fructose_100g | -gamma-linolenic-acid_100g | -glucose_100g | -gondoic-acid_100g | -lactose_100g | -lauric-acid_100g | -lignoceric-acid_100g | -linoleic-acid_100g | -maltodextrins_100g | -maltose_100g | -mead-acid_100g | -melissic-acid_100g | -montanic-acid_100g | -myristic-acid_100g | -nervonic-acid_100g | -oleic-acid_100g | -palmitic-acid_100g | -stearic-acid_100g | -sucrose_100g | additives | allergens_en | cities | no_nutriments | 

37

> A ce stade, si l'on supprime les colonnes totalement vides, on a donc un jeu de données qui passe de 186 à **149** colonnes.

In [98]:
cols_to_keep = set(prob.columns) - set(cols_fillrate_00)

size = max([len(x) for x in cols_to_keep])
for i, col in enumerate(sorted(cols_to_keep)):
    end_sep="\t" if i%2 == 0 else "\n"
    print(f"{col[:size].ljust(size)}", end=end_sep)

abbreviated_product_name                             	additives_en                                         
additives_n                                          	additives_tags                                       
alcohol_100g                                         	allergens                                            
beta-carotene_100g                                   	beta-glucan_100g                                     
bicarbonate_100g                                     	biotin_100g                                          
brand_owner                                          	brands                                               
brands_tags                                          	caffeine_100g                                        
calcium_100g                                         	carbohydrates_100g                                   
carbon-footprint-from-meat-or-fish_100g              	carbon-footprint_100g                                
carnitine_100g              

#### Supprimons les colonnes redondantes

Notons que les colonnes redondantes pourraient servir pour les imputations des colonnes gardées, mais on reviendra dessus si nécessaire.

In [99]:
cols_redondant = set([
    'additives_en', # 'additives_tags'
    'created_datetime', # --> 'created_t'
    'last_modified_datetime', # --> 'last_modified_t'
    'generic_name', # --> 'product_name'
    'abbreviated_product_name', # --> 'product_name'
    'packaging_tags', # --> 'packaging'
    'packaging_en', # --> 'packaging'
    'brands', # --> 'brands_tags'
    'categories', #--> 'categories_tags'
    'categories_en',  #--> 'categories_tags'
    'manufacturing_places', # --> 'manufacturing_places_tags'
    'labels', # --> 'labels_tags'
    'labels_en', # --> 'labels_tags'
    'emb_codes', # --> 'emb_codes_tags'
    'countries', # --> 'countries_tags'
    'countries_en', # --> 'countries_tags'
    'ingredients_text', # --> 'ingredients_tags'
    'traces_en', # --> 'traces_tags'
    'additives_en', # --> 'additives_tags' 
    'food_groups_tags', # --> 'food_groups'
    'food_groups_en', # --> 'food_groups'
    'states', # --> 'states_tags'
    'states_en', # --> 'states_tags'
    'main_category_en', # --> 'main_category'
    #'energy-kj_100g', # --> 'energy_100g' # je la garde pour imputation
    #'energy-kcal_100g', # --> 'energy_100g' # je la garde pour imputation
    'fruits-vegetables-nuts-dried_100g', # --> 'fruits-vegetables-nuts-estimate-from-ingredients_100g'
    'fruits-vegetables-nuts-estimate_100g', # --> 'fruits-vegetables-nuts-estimate-from-ingredients_100g'
    'fruits-vegetables-nuts_100g', # --> 'fruits-vegetables-nuts-estimate-from-ingredients_100g'
    #'nutriscore_score', # --> 'nutriscore_grade'
    #'nutrition-score-fr_100g', # --> 'nutriscore_score'
    #'nutrition-score-uk_100g', # --> 'nutriscore_score'
    'origins', # --> 'origins_tags'
    'origins_en', # --> 'origins_tags'
    'packaging', # --> 'packaging_text'
    'traces', # --> 'traces_tags'
])

In [100]:
cols_to_keep -= set(cols_redondant)

size = max([len(x) for x in cols_to_keep])
for i, col in enumerate(sorted(cols_to_keep)):
    end_sep="\t" if i%2 == 0 else "\n"
    print(f"{col[:size].ljust(size)}", end=end_sep)
    
len(cols_to_keep)

additives_n                                          	additives_tags                                       
alcohol_100g                                         	allergens                                            
beta-carotene_100g                                   	beta-glucan_100g                                     
bicarbonate_100g                                     	biotin_100g                                          
brand_owner                                          	brands_tags                                          
caffeine_100g                                        	calcium_100g                                         
carbohydrates_100g                                   	carbon-footprint-from-meat-or-fish_100g              
carbon-footprint_100g                                	carnitine_100g                                       
casein_100g                                          	categories_tags                                      
chloride_100g               

119

> A ce stade, si l'on supprime les colonnes redondantes en plus des colonnes totalement vides, on a donc un jeu de données qui passe de 186 à **119** colonnes.

#### Supprimons les colonnes ne présentant a-priori pas d'intérêt d'un point de vue sanitaire

In [101]:
cols_useless = set([
    'brand_owner',
    'brands_tags',
    'countries_tags',
    'cities_tags',
    'created_t',
    'creator',
    'emb_codes_tags',
    'food_groups',
    'first_packaging_code_geo',
    'image_ingredients_small_url',
    'image_ingredients_url',
    'image_nutrition_small_url',
    'image_nutrition_url',
    'image_small_url',
    'image_url',
    'last_modified_t',
    'manufacturing_places_tags',
    'origins_tags',
    'stores',
    'packaging_text',
    'purchase_places',
    #'code', # on garde
    #'url', # on garde
    #'product_name', #on garde
])

In [102]:
cols_to_keep -= set(cols_useless)

size = max([len(x) for x in cols_to_keep])
for i, col in enumerate(sorted(cols_to_keep)):
    end_sep="\t" if i%2 == 0 else "\n"
    print(f"{col[:size].ljust(size)}", end=end_sep)
    
len(cols_to_keep)

additives_n                                          	additives_tags                                       
alcohol_100g                                         	allergens                                            
beta-carotene_100g                                   	beta-glucan_100g                                     
bicarbonate_100g                                     	biotin_100g                                          
caffeine_100g                                        	calcium_100g                                         
carbohydrates_100g                                   	carbon-footprint-from-meat-or-fish_100g              
carbon-footprint_100g                                	carnitine_100g                                       
casein_100g                                          	categories_tags                                      
chloride_100g                                        	chlorophyl_100g                                      
cholesterol_100g            

98

> A ce stade, si l'on supprime les colonnes "inutiles", plus les colonnes redondantes et les colonnes totalement vides, on a donc un jeu de données qui passe de 186 à **98** colonnes.
>
> Sur ces 98 colonnes:
> - 76 sont des données nutritionnelles *(pour 100g)*
> - 5 sont des scores *(ecoscore_score, ecoscore_grade, nutriscore_score, nutriscore_grade, nova_group)*
> - 4 sont des listes d'ingrédients *(ingredients_tags, allergens, trace_tags, additive_tags)*
> - 5 sont des données de catégorisation *(pnns_group1, pnns_group2, main_category, categories_tags, labels_tags)*
> - 4 sont des données de quantification *(serving_size, serving_quantity, quantity, additives_n)*
> - 4 sont des méta-données *(code, product_name, url, states_tags)*

#### A présent regardons les colonnes avec un très faible taux de remplissage

Partons du principe que l'on retire toutes les colonnes dont le taux de remplissage est inférieur à 1%.

In [214]:
min_rate = 0.01 # entre 0 et 1

fill_ratio_low = {}
for col in cols_to_keep:
    ratio = fill_ratio[col]
    if ratio < min_rate:
        fill_ratio_low[col] = fill_ratio[col]
fill_ratio_low = pd.Series(fill_ratio_low)

In [215]:
size = max([len(x) for x in prob.columns])-13

for i, ratio in enumerate(fill_ratio_low.iteritems()):
    end_sep="\t" if i%2 == 0 else "\n"
    print(f"{i:3} {ratio[0][:size].ljust(size)} {ratio[1]*100:7.2f}%", end=end_sep)

  0 cocoa_100g                                  0.30%	  1 iodine_100g                                 0.11%
  2 water-hardness_100g                         0.00%	  3 energy-from-fat_100g                        0.04%
  4 vitamin-d_100g                              0.46%	  5 biotin_100g                                 0.06%
  6 chromium_100g                               0.01%	  7 nucleotides_100g                            0.00%
  8 carbon-footprint_100g                       0.02%	  9 beta-glucan_100g                            0.00%
 10 omega-9-fat_100g                            0.01%	 11 zinc_100g                                   0.46%
 12 choline_100g                                0.00%	 13 insoluble-fiber_100g                        0.15%
 14 polyols_100g                                0.17%	 15 copper_100g                                 0.19%
 16 beta-carotene_100g                          0.00%	 17 bicarbonate_100g                            0.02%
 18 vitamin-b6_100g         

In [220]:
cols_low_rate = fill_ratio_low.index

In [221]:
cols_to_keep -= set(cols_low_rate)

size = max([len(x) for x in cols_to_keep])
for i, col in enumerate(sorted(cols_to_keep)):
    end_sep="\t" if i%2 == 0 else "\n"
    print(f"{col[:size].ljust(size)}", end=end_sep)
    
len(cols_to_keep)

additives_n                                          	additives_tags                                       
alcohol_100g                                         	allergens                                            
calcium_100g                                         	carbohydrates_100g                                   
categories_tags                                      	cholesterol_100g                                     
code                                                 	ecoscore_grade                                       
ecoscore_score                                       	energy-kcal_100g                                     
energy-kj_100g                                       	energy_100g                                          
fat_100g                                             	fiber_100g                                           
fruits-vegetables-nuts-estimate-from-ingredients_100g	ingredients_tags                                     
iron_100g                   

48

> A ce stade, si l'on supprime les colonnes avec moins de 1% de remplissage plus les colonnes "inutiles", les colonnes redondantes et les colonnes totalement vides, on a donc un jeu de données qui passe de 186 à **48** colonnes.
>
> Sur ces 48 colonnes:
> - 26 sont des données nutritionnelles *(pour 100g)*
> - 5 sont des scores *(ecoscore_score, ecoscore_grade, nutriscore_score, nutriscore_grade, nova_group)*
> - 4 sont des listes d'ingrédients *(ingredients_tags, allergens, trace_tags, additive_tags)*
> - 5 sont des données de catégorisation *(pnns_group1, pnns_group2, main_category, categories_tags, labels_tags)*
> - 4 sont des données de quantification *(serving_size, serving_quantity, quantity, additives_n)*
> - 4 sont des méta-données *(code, product_name, url, states_tags)*

# 4. Créations d'un jeu de données réduit

In [224]:
data_low = pd.read_csv(filename, sep='\t', dtype=dtypes, usecols=cols_to_keep)

In [225]:
data_low.shape

(2251894, 48)

In [232]:
data_low.to_csv('data/data_low_cols.csv', index=False)

### La suite du nettoyage prend place dans le second notebook en partant de ce jeu de données réduit.